In [2]:
import tensorflow as tf
import pano_generator

model = tf.keras.models.load_model("../final_model_without_aug/")


In [7]:
inputs = tf.keras.Input([260,260,3])
output = model(inputs)[:,1]
new = tf.keras.Model(inputs, output, name="Pano")
new.compile('Adam',loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])

In [10]:
train_df,val_df,df = pano_generator.get_csv()

def mapping(path,label):

    image = tf.io.read_file(path)
    image = tf.io.decode_jpeg(image, channels=3)
    return image, label

path = f"../Pano_Data/test/"
imgs = df["key"] + ".jpg"

image_paths = path + imgs

labels = df["pano"].apply(int)

ds = tf.data.Dataset.from_tensor_slices((image_paths,labels))

ds = ds.map(mapping)
ds = ds.batch(batch_size=128)
ds = ds.prefetch(tf.data.AUTOTUNE)

In [12]:
preds = new.predict(ds)

In [16]:
((df.pano == True) == (preds > 0.5)).mean()

0.9991697799916978

In [17]:
new.save('../deploy_model')

INFO:tensorflow:Assets written to: ../deploy_model\assets
